In [25]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

In [26]:
df = pd.read_csv("stackoverflow_data100000.csv")
df3 = df.copy(deep=True)
df.head()

,Unnamed: 0,Body,Title,Tags,Code
0,0,want use trackbar change form opacity code dec...,convert decimal to double,c# floating-point type-conversion double ...,"[' track-bar ', ' form ', ' decimal trans ..."
1,1,absolutely positioned div containing several c...,percentage width child element in absolutely p...,html css internet-explorer-7,"[' div ', ' div ', ' percentage-based width ',..."
2,3,given datetime representing person birthday ca...,how do i calculate someone's age in c#?,c# .net datetime,[' datetime ']
3,4,given specific datetime value display relative...,calculate relative time in c#,c# datetime time datediff relative-time-s...,[' user-agent ']
4,6,standard way web server able determine user ti...,determine a user's timezone,html browser timezone user-agent timezone...,"[' dataset ', ' datatable ', ' dataset ', ' da..."


In [27]:
df.drop(["Unnamed: 0" , 'Code'] , axis=1 , inplace=True)

In [28]:
df.isnull().sum()

Body     2
Title    0
Tags     0
dtype: int64

In [29]:
df.head(3)

,Body,Title,Tags
0,want use trackbar change form opacity code dec...,convert decimal to double,c# floating-point type-conversion double ...
1,absolutely positioned div containing several c...,percentage width child element in absolutely p...,html css internet-explorer-7
2,given datetime representing person birthday ca...,how do i calculate someone's age in c#?,c# .net datetime


In [30]:
df['text'] = df['Tags']  + " " + df['Body'] 

In [31]:
df.head(2)

,Body,Title,Tags,text
0,want use trackbar change form opacity code dec...,convert decimal to double,c# floating-point type-conversion double ...,c# floating-point type-conversion double ...
1,absolutely positioned div containing several c...,percentage width child element in absolutely p...,html css internet-explorer-7,html css internet-explorer-7 absolutely po...


In [32]:
df = df[['text' , 'Tags']]

In [33]:
df.head(3)

,text,Tags
0,c# floating-point type-conversion double ...,c# floating-point type-conversion double ...
1,html css internet-explorer-7 absolutely po...,html css internet-explorer-7
2,c# .net datetime given datetime representi...,c# .net datetime


In [34]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [35]:
def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [36]:
#df['text'] = df['text'].apply(clean_txt)

In [37]:
df.dropna(inplace=True)
df.isnull().sum()

text    0
Tags    0
dtype: int64

In [134]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_df = tfidf_vectorizer.fit_transform((df['text'])) #fitting and transforming the vector
tfidf_df

<75225x306602 sparse matrix of type '<class 'numpy.float64'>'
	with 2993809 stored elements in Compressed Sparse Row format>

In [136]:
from sklearn.metrics.pairwise import cosine_similarity

## You can use any query word here and re-run the code

In [137]:
queries=["mysql"]

In [138]:
df2 = pd.DataFrame()
df2['queries']= queries

In [139]:
df2['queries']

0    mysql
Name: queries, dtype: object

# The top recommendations using TF-IDF

In [140]:
user_tfidf = tfidf_vectorizer.transform(df2['queries'])

In [141]:
cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_df)

In [142]:
output2 = list(cos_similarity_tfidf)

In [143]:
def get_recommendation(top, df3, scores):
    recommendation = pd.DataFrame(columns = ['Body', 'Title'])
    count = 0
    for i in top:
        recommendation.at[count, 'Body'] = df3['Body'][i]
        recommendation.at[count, 'Title'] = df3['Title'][i]
        recommendation.at[count, 'Tags'] = df3['Tags'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

In [120]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]

In [121]:
list_scores = [output2[i][0][0] for i in top]

In [122]:
get_recommendation(top,df3, list_scores)

,Body,Title,Tags,score
0,two translateanimations textview want execute ...,android animation one after other,android animation,0.749992
1,default charset mysql database mysql table mys...,how do i see what character set a mysql databa...,sql mysql unicode character-encoding coll...,0.709388
2,get current date format like 20122010,how do i get the current date?,iphone objective-c cocoa cocoa-touch date,0.706283
3,want develop project google app engine want us...,difference between jpa and jdo?,jpa jdo,0.659655
4,apache feature mysql one exist,how do you refresh the mysql configuration fil...,mysql database,0.650859
5,listsize 0 look implementation arraylist looks...,why is list.size()>0 slower than list.isempty(...,java list,0.649181
6,beginner question font handling cocoa font fam...,how do i create an nsfont that is both bold an...,cocoa fonts,0.646503
7,connect mysql database using python program,how do i connect to a mysql database in python?,python mysql,0.634346
8,custom title bar requestwindowfeaturewindowfea...,android: custom title bar,android layout titlebar,0.632392
9,trying connect mysql windows following instruc...,cannot add connection to mysql in c# application,c# mysql,0.631517


## The top recommendations using CountVectorizer

In [144]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

count_df = count_vectorizer.fit_transform((df['text'])) #fitting and transforming the vector
count_df

<75225x306602 sparse matrix of type '<class 'numpy.int64'>'
	with 2993809 stored elements in Compressed Sparse Row format>

In [145]:
query_count = count_vectorizer.transform(df2['queries'])
cos_similarity_countv = map(lambda x: cosine_similarity(query_count, x),count_df)

In [146]:
output2 = list(cos_similarity_countv)

In [147]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]

In [148]:
list_scores = [output2[i][0][0] for i in top]

In [149]:
get_recommendation(top,df3, list_scores)

,Body,Title,Tags,score
0,default charset mysql database mysql table mys...,how do i see what character set a mysql databa...,sql mysql unicode character-encoding coll...,0.784465
1,beginner question font handling cocoa font fam...,how do i create an nsfont that is both bold an...,cocoa fonts,0.771517
2,already crossposted ms connect attempting over...,overriding msbuildextensionspath in the msbuil...,msbuild msbuild-task,0.755929
3,get current date format like 20122010,how do i get the current date?,iphone objective-c cocoa cocoa-touch date,0.742781
4,set default value mysql datetime column sql se...,how do you set a default value for a mysql dat...,mysql datetime,0.718421
5,listsize 0 look implementation arraylist looks...,why is list.size()>0 slower than list.isempty(...,java list,0.717137
6,generic question search best answer would like...,how to implement a network protocol?,java c++ protocols,0.707894
7,advise installing logdata files mysql actually...,"mysql install the log,data and other files",mysql,0.707107
8,two translateanimations textview want execute ...,android animation one after other,android animation,0.707107
9,lost mysql username password retrieve,how do i retrieve my mysql username and password?,mysql,0.707107


In [103]:
from sklearn.neighbors import NearestNeighbors
n_neighbors = 10
KNN = NearestNeighbors(n_neighbors, p=2)
KNN.fit(tfidf_df)
NNs = KNN.kneighbors(user_tfidf, return_distance=True)

In [104]:
NNs[0][0][1:]

array([0.66615627, 0.68863144, 0.68958106, 0.70628753, 0.77966747,
       0.78990228, 0.8118189 , 0.82225982, 0.83759344])

In [105]:
top = NNs[1][0][1:]
index_score = NNs[0][0][1:]

get_recommendation(top, df3, index_score)

,Body,Title,Tags,score
0,every decent php programmer printr vardump wra...,a more pretty/informative var_dump alternative...,php debugging variables var-dump,0.666156
1,get time today time date datehis strtotimetoda...,time calculation in php (add 10 hours)?,php time addition,0.688631
2,got job offer today position sharepoint develo...,how good/bad is sharepoint programming?,c# .net sharepoint,0.689581
3,heard microsoft sharepoint used many companies...,what is microsoft sharepoint?,sharepoint,0.706288
4,escapejavascript method actionview escapes apo...,rails escape_javascript creates invalid json b...,ruby-on-rails json escaping,0.779667
5,lets say value 34679 want 346 truncate two dec...,truncate two decimal places without rounding,c# math rounding,0.789902
6,nowadays read much code injection exploits buf...,is code injection possible in java?,java code-injection,0.811819
7,way create sharepoint calculated column return...,can i create a countif calculated column in sh...,sharepoint sharepoint-2007,0.822260
8,steps required attaching event receiver custom...,how to attach an event receiver to a custom li...,c# sharepoint,0.837593


# Wordcloud

In [ ]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
bunch_text = " ".join(text for text in df.tail(10000).text)

In [ ]:
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color="white", colormap= "magma").generate(bunch_text)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[11,11])
plt.imshow(wordcloud, interpolation="sinc")
plt.axis("off")
plt.show()